In [33]:
import pandas as pd
import json
import requests
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from geopandas.tools import sjoin
import gmplot
import numpy as np
from shapely.geometry import Point, Polygon
import fiona
from osgeo import gdal
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
import elevation 
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default

In [34]:
data_2019 = pd.read_csv("data_2019_backup.csv")

In [35]:
data_2019.shape

(18382, 20)

In [36]:
data_2019["CUSTOMER_ID"].unique().shape

(3841,)

In [39]:
campus_buildings = gpd.read_file("campus_buildings.kml") #shape files used for campus buildings

In [42]:
#create useful util dict that does from id to building name
id_to_building = {}
for index, row in campus_buildings.iterrows():
    id_to_building[row["BUILDINGID"]] = row["Name"]

In [46]:
start_onc_2019 = data_2019[data_2019["start_on_campus"] == True]
start_and_end_onc_2019 = start_onc_2019[start_onc_2019["end_on_campus"]]
start_onc_end_off_2019 = start_onc_2019[start_onc_2019["end_on_campus"] == False]

start_off_end_on_2019 = data_2019[data_2019["start_on_campus"] == False]
start_off_end_on_2019 = start_off_end_on_2019[start_off_end_on_2019["end_on_campus"]]

all_off_2019 = data_2019[data_2019["start_on_campus"] == False]
all_off_2019 = all_off_2019[all_off_2019["end_on_campus"] == False]

In [48]:
def filter_outliers(df, col):
    q_low = df[col].quantile(0.01)
    q_hi  = df[col].quantile(0.99)

    return df[(df[col] < q_hi) & (df[col] > q_low)]

In [49]:
start_and_end_onc_2019 = filter_outliers(start_and_end_onc_2019, "DISTANCE")
start_and_end_onc_2019["DISTANCE"].mean()

0.7663758379037172

In [50]:
start_onc_end_off_2019 = filter_outliers(start_onc_end_off_2019, "DISTANCE")
start_onc_end_off_2019["DISTANCE"].mean()

0.8390943744236091

In [51]:
start_off_end_on_2019 = filter_outliers(start_off_end_on_2019, "DISTANCE")
start_off_end_on_2019["DISTANCE"].mean()

0.8075382253607615

In [52]:
all_off_2019 = filter_outliers(all_off_2019, "DISTANCE")
all_off_2019["DISTANCE"].mean()

0.9915413755057007

In [54]:
start_and_end_onc_2019 = pd.read_csv("start_and_end_onc_2019.csv") #precomputed values

In [56]:
#First, let's go by start and end destination combination
def helper(row):
    return str(row["start_build"]) + "-" + str(row["end_build"])
start_and_end_onc_2019["start-end"] = start_and_end_onc_2019.apply(helper, axis=1)

In [57]:
#Visualization Methods
def get_coors_based_on_start_end_return_one_df(df, combo):
    df["start-end"] = df.apply(helper, axis=1)
    df = df[df["start-end"] == combo]
    return get_coors(df)
def get_coors_based_on_end(df, build_id):
    df = df[df["end_build"] == int(build_id)]
    print(df.shape)
    return get_coors(df)
def get_coors(df):
    latitude = []
    longitude = []
    for idx, row in df.iterrows():
        coors = json.loads(row["PATH"])["coordinates"]
        for i in coors:
            longitude.append(i[1])
            latitude.append(i[0])
    ret = pd.DataFrame()
    ret[0] = np.array(latitude)
    ret[1] = np.array(longitude)
    return ret
def create_drawer():
    on_campus_boxes = [[38.993558, -76.950699, 38.982917, -76.933831]]
    b = on_campus_boxes[0]
    min_lat, max_lat, min_lon, max_lon = b[2], b[0], b[1], b[3]
    ## Create empty map with zoom level 16
    mymap = gmplot.GoogleMapPlotter(
        min_lat + (max_lat - min_lat) / 2, 
        min_lon + (max_lon - min_lon) / 2, 
        16, apikey="API KEY")
    return mymap
def write_drawer(mymap, df, name):
    mymap.heatmap(df[1], df[0])
    mymap.draw(name)

In [59]:
"""Start locations and parking locations"""
parking_spots = pd.read_csv("CPK_Parking_and_Restricted_Zones.csv")
#visually see all the starting locations
drawer_all_starts = create_drawer()
for idx, row in parking_spots.iterrows():
    x = json.loads(row["FENCECOORDINATES"])['coordinates'][0][2]
    drawer_all_starts.marker(x[1], x[0], title="Parking Location", color="blue")
new_df = pd.DataFrame()
new_df[0] = start_and_end_onc_2019["START LAT"]
new_df[1] = start_and_end_onc_2019["START LONG"]
drawer_all_starts.heatmap(new_df[0], new_df[1])
drawer_all_starts.draw("start_locations_heatmap_2019.html")

In [60]:
"""End locations and parking locations"""
parking_spots = pd.read_csv("CPK_Parking_and_Restricted_Zones.csv")
#visually see all the starting locations
drawer_all_starts = create_drawer()
for idx, row in parking_spots.iterrows():
    x = json.loads(row["FENCECOORDINATES"])['coordinates'][0][2]
    drawer_all_starts.marker(x[1], x[0], title="Parking Location", color="blue")
new_df = pd.DataFrame()
new_df[0] = start_and_end_onc_2019["END LAT"]
new_df[1] = start_and_end_onc_2019["END LONG"]
drawer_all_starts.heatmap(new_df[0], new_df[1])
drawer_all_starts.draw("end_locations_heatmap_2019.html")

In [162]:
"""Off-on"""
parking_spots = pd.read_csv("CPK_Parking_and_Restricted_Zones.csv")
#visually see all the starting locations
drawer_all_starts = create_drawer()
for idx, row in parking_spots.iterrows():
    x = json.loads(row["FENCECOORDINATES"])['coordinates'][0][2]
    drawer_all_starts.marker(x[1], x[0], title="Parking Location", color="blue")
new_df = pd.DataFrame()
new_df[0] = start_off_end_on_2019["START LAT"]
new_df[1] = start_off_end_on_2019["START LONG"]
drawer_all_starts.heatmap(new_df[0], new_df[1])
drawer_all_starts.draw("off_on_2019.html")

In [58]:
#Remove repeats from path
def parse_path(path):
    ex_path_arr = path.split("-")
    n_path = ""
    for i, curr_path in enumerate(ex_path_arr):
        if i != 0:
            if ex_path_arr[i - 1] != curr_path:
                n_path += curr_path + "-"
        else:
            n_path += curr_path + "-"
    return n_path[:-1]
start_and_end_onc_2019["path"] = start_and_end_onc_2019["path"].apply(parse_path)

202-202    248
48-68      192
202-68     181
48-48      176
68-48      172
68-68      166
68-202     165
Name: start-end, dtype: int64

In [65]:
start_and_end_onc_2019["start_build"].value_counts().head(5)

202    1024
68     1021
48      906
233     806
4       307
Name: start_build, dtype: int64

In [66]:
start_and_end_onc_2019["end_build"].value_counts().head(5)

68     1064
202     918
48      829
233     688
4       327
Name: end_build, dtype: int64

In [67]:
#create useful util dict that does from id to building name
id_to_building = {}
for index, row in campus_buildings.iterrows():
    id_to_building[row["BUILDINGID"]] = row["Name"]

In [88]:
id_to_building["004"]

'Ritchie Coliseum'

In [89]:
#Common Paths
oct_2019_onc_end_gym = start_and_end_onc_2019[start_and_end_onc_2019["end_build"] == 68]
oct_2019_onc_end_pf = start_and_end_onc_2019[start_and_end_onc_2019["end_build"] == 4]

In [90]:
common_paths_gym = list(oct_2019_onc_end_gym["path"].value_counts().index[1:15])
common_paths_pf = list(oct_2019_onc_end_pf["path"].value_counts().index[1:20])

In [91]:
def h2(path):
    return path in common_paths_gym
def h3(path):
    return path in common_paths_pf
oct_2019_onc_end_gym["common_path"] = oct_2019_onc_end_gym["path"].apply(h2)
oct_2019_onc_end_pf["common_path"] = oct_2019_onc_end_pf["path"].apply(h3)

/Users/abhi/opt/anaconda3/envs/veo/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/abhi/opt/anaconda3/envs/veo/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [92]:
comm_path_gym = oct_2019_onc_end_gym[oct_2019_onc_end_gym["common_path"] == True]
comm_path_pf = oct_2019_onc_end_pf[oct_2019_onc_end_pf["common_path"] == True]

In [85]:
#Finally, draw the examples of the common path
drawer = create_drawer()
for idx, row in comm_path_gym.iterrows():
    drawer.marker(row["START LAT"], row["START LONG"], title="start", color="green")
    drawer.marker(row["END LAT"], row["END LONG"], title="start", color="red")
df= get_coors(comm_path_gym)
write_drawer(drawer, df, "gym_common_paths_2019.html")

In [93]:
drawer = create_drawer()
for idx, row in comm_path_pf.iterrows():
    drawer.marker(row["START LAT"], row["START LONG"], title="start", color="green")
    drawer.marker(row["END LAT"], row["END LONG"], title="start", color="red")
df= get_coors(comm_path_pf)
write_drawer(drawer, df, "ritchie_common_paths_2019.html")

In [94]:
#Important constants and bbox information for on/off campus split paths
#Commercial Areas
comm_names = ["Baltimore Ave.", "Golf", "Whole_Foods", "Amazon_Locker"]
comm_bboxes = [[38.982787, -76.939088, 38.979837, -76.937430], [39.007407, -76.932781, 39.002550, -76.929479], [38.971380, -76.938418, 38.968577, -76.936229], [38.981258, -76.943836, 38.980094, -76.942471]]
#Residential Areas
res_names = ["University View", "The Varsity", "Beywyn", "Domain"]
res_bboxes = [[38.993135, -76.933713, 38.992230, -76.932447], [38.990749, -76.935151, 38.989939, -76.934585], [38.996936, -76.927787, 38.993408, -76.925937], [38.985241, -76.949473, 38.984269, -76.948507]]
#Metro/Bus Stops
metro_names = ["College Park Metro Station"]
metro_bboxes = [[38.978928, -76.929447, 38.977594, -76.928160]]

In [95]:
data_2019_on_off = start_onc_end_off_2019
#Determine the type of ending location
def in_bbox(box, lat, lon):
    if box[2] < lat and box[0] > lat:
        if box[1] < lon and lon < box[3]:
            return True
    return False
def in_res_area_helper(row):
    ret = "None"
    for i, bbox in enumerate(res_bboxes):
        if (in_bbox(bbox, row["END LAT"], row["END LONG"])):
            ret = res_names[i]
    return ret
def in_metro_helper(row):
    ret = "None"
    for i, bbox in enumerate(metro_bboxes):
        if (in_bbox(bbox, row["END LAT"], row["END LONG"])):
            ret = metro_names[i]
    return ret
def in_comm_helper(row):
    ret = "None"
    for i, bbox in enumerate(comm_bboxes):
        if (in_bbox(bbox, row["END LAT"], row["END LONG"])):
            ret = comm_names[i]
    return ret
data_2019_on_off["res_end"] = data_2019_on_off.apply(in_res_area_helper, axis=1)
data_2019_on_off["comm_end"] = data_2019_on_off.apply(in_comm_helper, axis=1)
data_2019_on_off["metro_end"] = data_2019_on_off.apply(in_metro_helper, axis=1)

In [134]:
data_2019_on_off["res_end"].value_counts()

None               2086
University View     603
Domain              481
Beywyn               58
The Varsity          25
Name: res_end, dtype: int64

In [139]:
data_2019_on_off["comm_end"].value_counts()

None              2502
Baltimore Ave.     671
Golf                51
Whole_Foods         26
Amazon_Locker        3
Name: comm_end, dtype: int64

In [140]:
data_2019_on_off["metro_end"].value_counts()

None                          2997
College Park Metro Station     256
Name: metro_end, dtype: int64

In [135]:
data_2019_on_off_end_balt = data_2019_on_off[data_2019_on_off["res_end"]== "University View"]

In [167]:
data_2019_on_off_end_balt = all_off_2019.sample(50)#.iloc[[9, 34, 139, 214, 267, 64, 228, 177, 274]]
drawer = create_drawer()
for idx, row in data_2019_on_off_end_balt.iterrows():
    drawer.marker(row["START LAT"], row["START LONG"], title="start", color="green")
    drawer.marker(row["END LAT"], row["END LONG"], title="start", color="red")
    path = json.loads(row["PATH"])
    path_df = pd.DataFrame(path["coordinates"])
    drawer.plot(path_df[1], path_df[0], 'blue', edge_width=1)
#new_df = pd.DataFrame()
#new_df[0] = data_2019_on_off_end_uv["START LAT"]
#new_df[1] = data_2019_on_off_end_uv["START LONG"]
#drawer.heatmap(new_df[0], new_df[1])
df= get_coors(data_2019_on_off)
#write_drawer(drawer, df, "on_off_paths.html")
drawer.draw("all_off_path.html")

In [ ]:
start_off_end_on_2019

In [136]:
data_2019_on_off_end_balt

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,CREATED,START,END,DISTANCE,RIDE_ID,CUSTOMER_ID,MINUTES,...,START LONG,START LAT,END LONG,END LAT,year,start_on_campus,end_on_campus,res_end,comm_end,metro_end
64,303,303,303,2019-10-28 20:28:24+00:00,2019-10-28 20:28:24+00:00,2019-10-28 20:33:10+00:00,0.750,1623362,262831,5,...,-76.946145,38.993266,-76.933364,38.992909,2019,True,False,University View,None,None
168,407,407,407,2019-10-15 19:32:51+00:00,2019-10-15 19:32:51+00:00,2019-10-15 19:42:28+00:00,0.921,1496643,243964,10,...,-76.937304,38.985537,-76.932833,38.992793,2019,True,False,University View,None,None
177,416,416,416,2019-10-01 16:05:16+00:00,2019-10-01 16:05:16+00:00,2019-10-01 16:10:29+00:00,0.585,1326246,195602,6,...,-76.940612,38.988611,-76.933327,38.992820,2019,True,False,University View,None,None
227,466,466,466,2019-10-29 14:35:10+00:00,2019-10-29 14:35:10+00:00,2019-10-29 14:39:13+00:00,0.448,1629475,273896,5,...,-76.940653,38.990259,-76.933315,38.992779,2019,True,False,University View,None,None
274,513,513,513,2019-10-29 20:49:02+00:00,2019-10-29 20:49:02+00:00,2019-10-29 20:56:13+00:00,1.336,1633399,220262,8,...,-76.943627,38.982432,-76.933363,38.992915,2019,True,False,University View,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18252,38256,38256,38256,2019-10-29 19:17:42+00:00,2019-10-29 19:17:42+00:00,2019-10-29 19:22:10+00:00,0.559,1632449,117446,5,...,-76.940819,38.989265,-76.933419,38.992891,2019,True,False,University View,None,None
18269,38273,38273,38273,2019-10-19 23:11:51+00:00,2019-10-19 23:11:51+00:00,2019-10-19 23:26:00+00:00,1.378,1542745,278383,15,...,-76.948227,38.984383,-76.933465,38.992875,2019,True,False,University View,None,None
18270,38274,38274,38274,2019-10-31 15:05:29+00:00,2019-10-31 15:05:29+00:00,2019-10-31 15:28:08+00:00,1.541,1645115,256366,23,...,-76.940821,38.989815,-76.933367,38.992800,2019,True,False,University View,None,None
18300,38304,38304,38304,2019-10-07 17:03:56+00:00,2019-10-07 17:03:56+00:00,2019-10-07 17:08:18+00:00,0.517,1401241,213026,5,...,-76.940880,38.990000,-76.933188,38.992537,2019,True,False,University View,None,None


In [103]:
drawer = create_drawer()
new_df = pd.DataFrame()
new_df[0] = data_2019_on_off["END LAT"]
new_df[1] = data_2019_on_off["END LONG"]
drawer.heatmap(new_df[0], new_df[1])
drawer.draw("on_off_end_2019.html")

In [141]:
start_onc_end_off_2019.shape

(3253, 23)

In [146]:
from datetime import datetime
def remove_end(time):
    return time.split("+")[0]
def get_day(time):
    return datetime.strptime(time, '%Y-%m-%d %H:%M:%S').day
def get_hour(time):
    return datetime.strptime(time, '%Y-%m-%d %H:%M:%S').hour

def perform_time_transform(df):
    df["start_time"] = df["START"].apply(remove_end)
    df["end_time"] = df["START"].apply(remove_end)
    df["hour"] = df["end_time"].apply(get_hour)
    df["day"] = df["end_time"].apply(get_day)
    return df

In [150]:
start_and_end_onc_2019 = perform_time_transform(start_and_end_onc_2019)
start_onc_end_off_2019 = perform_time_transform(start_onc_end_off_2019)
start_off_end_on_2019 = perform_time_transform(start_off_end_on_2019)
all_off_2019 = perform_time_transform(all_off_2019)

In [151]:
start_onc_end_off_2019["MINUTES"].mean()

10.788502920381186

In [152]:
start_and_end_onc_2019["MINUTES"].mean()

9.66319864505202

In [153]:
start_off_end_on_2019["MINUTES"].mean()

9.319619281547435

In [154]:
all_off_2019["MINUTES"].mean()

12.991541007723427

In [172]:
data_2019_on_off["res_end"].value_counts()

None               2086
University View     603
Domain              481
Beywyn               58
The Varsity          25
Name: res_end, dtype: int64

In [ ]:
data_2019_on_off["res_end"]

In [176]:
data_2019_on_off[data_2019_on_off["res_end"] != "None"]["hour"].value_counts()

16    130
18    128
19    113
17    103
14     91
22     90
20     83
15     83
21     81
23     79
13     71
0      60
12     25
11     22
9       7
1       1
Name: hour, dtype: int64

In [177]:
data_2019_on_off["hour"].value_counts()

18    338
16    333
19    317
17    315
20    289
22    286
23    268
21    236
0     218
15    215
14    182
13    120
12     62
11     48
9      14
10      9
1       3
Name: hour, dtype: int64